<a href="https://colab.research.google.com/github/yakovsushenok/Thesis/blob/main/Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import zipfile
import os
import pandas as pd
import math, random
import torch
import torchaudio
from torchaudio import transforms
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import init
from google.colab import drive
from sklearn.model_selection import StratifiedShuffleSplit
drive.mount('/content/gdrive')
import time
import matplotlib.pyplot as plt
import numpy as np
import numpy.matlib
try:
    from scipy.fftpack import fft, ifft
except ImportError:
    from numpy.fft import fft, ifft
from scipy.signal import lfilter
import scipy.io as sio
from scipy import signal
import gc
import h5py
from torchsummary import summary
!pip install torchmetrics
from torchmetrics.functional import f1_score
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
df_500 = pd.read_csv("/content/gdrive/MyDrive/train_metadata_more_than_500.csv")
df_500['primary_label'] = df_500['primary_label'].apply(lambda x: x - 1)
df_500 = df_500[['relative_path', 'primary_label']]
df_500_toy = pd.read_csv("/content/gdrive/MyDrive/df_500_toy.csv")
toy_ind = list(df_500_toy['Unnamed: 0'])

class H5DS(Dataset):
  def __init__(self, df, path, toy_ind):
    self.path = path
    self.data = h5py.File(self.path, 'r')['data']
    self.df = df
    self.toy_ind = toy_ind
  
  def __len__(self):
    return len(self.df)    
    
  def __getitem__(self, idx):
   
   return (self.data[toy_ind[idx]], torch.tensor(self.df['primary_label'].iloc[idx]))

device = torch.device("cuda:0" if torch.cuda.is_available() else  "cpu") #

def inference(model, val_dl):
  correct_prediction = 0
  total_prediction = 0
  outputsList = []
  labelsList = []
  # Disable gradient updates
  with torch.no_grad():
    for data in val_dl:
      # Get the input features and target labels, and put them on the GPU
      inputs, labels = data[0].to(device), data[1].to(device)

      # Normalizing
      inputs -= inputs.min(1, keepdim=True)[0]
      inputs /= inputs.max(1, keepdim=True)[0]

      inputs = inputs[None, :, :, :]
      inputs = inputs.permute(1, 0, 2, 3)
      inputs = torch.cat([inputs[:, :, :round(inputs.shape[2]/3), :],inputs[:, :, round(inputs.shape[2]/3):round(inputs.shape[2]*2/3), :],inputs[:, :, round(inputs.shape[2]*2/3):, : ]], dim = 1)

      # Get predictions
      outputs = model(inputs.float())

      # Get the predicted class with the highest score
      _, prediction = torch.max(outputs,1)
      # Count of predictions that matched the target label
      correct_prediction += (prediction == labels).sum().item()
      total_prediction += prediction.shape[0]
      outputsList.append(prediction.cpu().numpy())
      labelsList.append(labels.cpu().numpy())
  # Accuracy
  acc = correct_prediction/total_prediction
  print(f'Val Accuracy: {acc:.2f}')
  # F1 score
  outputs = np.concatenate(outputsList)
  targets = np.concatenate(labelsList)
  f1 = f1_score(torch.from_numpy(outputs),torch.from_numpy(targets), num_classes = 12, average='micro')
  print(f"F1 score: {f1}")
  return f1, acc
  




# Resnet Model

[Link](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html) for saving best model (how to do it)

https://pytorch.org/tutorials/beginner/saving_loading_models.html

In [13]:
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, downsample):
        super().__init__()
        if downsample:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1)
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
            self.shortcut = nn.Sequential()

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, input):
        shortcut = self.shortcut(input)
        input = nn.ReLU()(self.bn1(self.conv1(input)))
        input = nn.ReLU()(self.bn2(self.conv2(input)))
        input = input + shortcut
        return nn.ReLU()(input)

class ResBottleneckBlock(nn.Module):
    def __init__(self, in_channels, out_channels, downsample):
        super().__init__()
        self.downsample = downsample
        self.conv1 = nn.Conv2d(in_channels, out_channels//4, kernel_size=1, stride=1)
        self.conv2 = nn.Conv2d(out_channels//4, out_channels//4, kernel_size=3, stride=2 if downsample else 1, padding=1)
        self.conv3 = nn.Conv2d(out_channels//4, out_channels, kernel_size=1, stride=1)
        self.shortcut = nn.Sequential()

        if self.downsample or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2 if self.downsample else 1),
                nn.BatchNorm2d(out_channels)
            )

        self.bn1 = nn.BatchNorm2d(out_channels//4)
        self.bn2 = nn.BatchNorm2d(out_channels//4)
        self.bn3 = nn.BatchNorm2d(out_channels)

    def forward(self, input):
        shortcut = self.shortcut(input)
        input = nn.ReLU()(self.bn1(self.conv1(input)))
        input = nn.ReLU()(self.bn2(self.conv2(input)))
        input = nn.ReLU()(self.bn3(self.conv3(input)))
        input = input + shortcut
        return nn.ReLU()(input)

class ResNet(nn.Module):
    def __init__(self, in_channels, resblock, repeat, useBottleneck=False, outputs=12):
        super().__init__()
        self.layer0 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        if useBottleneck:
            filters = [64, 256, 512, 1024, 2048]
        else:
            filters = [64, 64, 128, 256, 512]

        self.layer1 = nn.Sequential()
        self.layer1.add_module('conv2_1', resblock(filters[0], filters[1], downsample=False))
        for i in range(1, repeat[0]):
                self.layer1.add_module('conv2_%d'%(i+1,), resblock(filters[1], filters[1], downsample=False))

        self.layer2 = nn.Sequential()
        self.layer2.add_module('conv3_1', resblock(filters[1], filters[2], downsample=True))
        for i in range(1, repeat[1]):
                self.layer2.add_module('conv3_%d' % (i+1,), resblock(filters[2], filters[2], downsample=False))

        self.layer3 = nn.Sequential()
        self.layer3.add_module('conv4_1', resblock(filters[2], filters[3], downsample=True))
        for i in range(1, repeat[2]):
            self.layer3.add_module('conv2_%d' % (i+1,), resblock(filters[3], filters[3], downsample=False))

        self.layer4 = nn.Sequential()
        self.layer4.add_module('conv5_1', resblock(filters[3], filters[4], downsample=True))
        for i in range(1, repeat[3]):
            self.layer4.add_module('conv3_%d'%(i+1,), resblock(filters[4], filters[4], downsample=False))

        self.gap = torch.nn.AdaptiveAvgPool2d(1)
        self.fc = torch.nn.Linear(filters[4], outputs)

    def forward(self, input):
        input = self.layer0(input)
        input = self.layer1(input)
        input = self.layer2(input)
        input = self.layer3(input)
        input = self.layer4(input)
        input = self.gap(input)
        input = torch.flatten(input, start_dim=1)
        input = self.fc(input)

        return input
class H5DS(Dataset):
  def __init__(self, df, path):
    self.path = path
    self.data = h5py.File(self.path, 'r')['data']
    self.df = df
    self.toy_ind = toy_ind
  
  def __len__(self):
    return len(self.df)    
    
  def __getitem__(self, idx):
   
   return (self.data[idx], torch.tensor(self.df['primary_label'].iloc[idx]))

device = torch.device("cuda:0" if torch.cuda.is_available() else  "cpu") #

myModel = ResNet(3, ResBlock, [2, 2, 2, 2], useBottleneck=False, outputs=12) # resnet18
device = torch.device("cuda:0" if torch.cuda.is_available() else  "cpu") #
myModel = myModel.to(device)




def training(model, train_dl, num_epochs, val_dl):
  # Loss Function, Optimizer 
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr= 10**(-5))
  
  val_accuracy_epoch_list = [] 
  train_accuracy_epoch_list = [] 
  best_f1 = 0
  epoch_f1 = 0
  # Epoch iterator
  for epoch in range(num_epochs):
    running_loss = 0.0
    correct_prediction = 0
    total_prediction = 0
    indices = []
    # Batch iterator
    for i, data in enumerate(train_dl):

        inputs, labels = torch.tensor(data[0]).to(device), torch.tensor(data[1]).to(device) # Get the input features and target labels, and put them on the GPU
        if torch.isnan(torch.tensor(data[0])).any() == True:
          continue
        
        # Normalize the inputs
        inputs -= inputs.min(1, keepdim=True)[0]
        inputs /= inputs.max(1, keepdim=True)[0]

        inputs = inputs[None, :, :, :]
        inputs = inputs.permute(1, 0, 2, 3)
        inputs = torch.cat([inputs[:, :, :round(inputs.shape[2]/3), :],inputs[:, :, round(inputs.shape[2]/3):round(inputs.shape[2]*2/3), :],inputs[:, :, round(inputs.shape[2]*2/3):, : ]], dim = 1)

        optimizer.zero_grad() # Zero the parameter gradients

        # forward + backward + optimize
        outputs = model(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() # Keep stats for Loss and Accuracy

        _, prediction = torch.max(outputs,1) # Get the predicted class with the highest score
        correct_prediction += (prediction == labels).sum().item() # Count of predictions that matched the target label
        total_prediction += prediction.shape[0]

    
    # Print stats at the end of the epoch
    num_batches = len(train_dl)
    avg_loss = running_loss / num_batches
    acc = correct_prediction/total_prediction
    train_accuracy_epoch_list.append(acc)
    print(f'Epoch: {epoch + 1}, Loss: {avg_loss:.2f}, Accuracy: {acc:.2f}')
    print(f"TESTING:")
    epoch_f1, val_acc = inference(model, val_dl)
    val_accuracy_epoch_list.append(val_acc)
    if epoch_f1 > best_f1:
      torch.save(model.state_dict(), "/content/gdrive/MyDrive/BestResnet18.pt")
      best_f1 = epoch_f1
      # model = TheModelClass(*args, **kwargs)
      # model.load_state_dict(torch.load(PATH))
      # model.eval()


    print("\n")

  print('Finished Training')
  return train_accuracy_epoch_list, val_accuracy_epoch_list


path = '/content/gdrive/MyDrive/df_train_tensor_60_fs50-8k.h5'                
NUM_EPOCHS = 30
# Initializing the dataset
myds = H5DS(df_500, path)
# Random split of 80:20 between training and validation
num_items = len(myds)
num_train = round(num_items*(0.80))
num_val = int((num_items - num_train)//2)
num_test = num_items - num_train - num_val
print(num_train,num_val, num_test)
train_ds, val_ds, test_ds = random_split(myds, [num_train, num_val, num_test])
# Create training and validation data loaders
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=1, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=1, shuffle=False)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=1, shuffle=False)

train_acc_list, val_acc_list = training(myModel, train_dl, NUM_EPOCHS, val_dl) # Training

4800 600 600


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:142: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:143: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch: 1, Loss: 2.30, Accuracy: 0.22
TESTING:
Val Accuracy: 0.34
F1 score: 0.34166666865348816


Epoch: 2, Loss: 1.84, Accuracy: 0.43
TESTING:
Val Accuracy: 0.48
F1 score: 0.48500001430511475


Epoch: 3, Loss: 1.51, Accuracy: 0.57
TESTING:
Val Accuracy: 0.56
F1 score: 0.5633333325386047


Epoch: 4, Loss: 1.26, Accuracy: 0.66
TESTING:
Val Accuracy: 0.67
F1 score: 0.6683333516120911


Epoch: 5, Loss: 1.07, Accuracy: 0.71
TESTING:
Val Accuracy: 0.69
F1 score: 0.6933333277702332


Epoch: 6, Loss: 0.91, Accuracy: 0.75
TESTING:
Val Accuracy: 0.78
F1 score: 0.778333306312561


Epoch: 7, Loss: 0.78, Accuracy: 0.79
TESTING:
Val Accuracy: 0.74
F1 score: 0.7383334040641785


Epoch: 8, Loss: 0.67, Accuracy: 0.82
TESTING:
Val Accuracy: 0.69
F1 score: 0.6916666626930237


Epoch: 9, Loss: 0.58, Accuracy: 0.85
TESTING:
Val Accuracy: 0.78
F1 score: 0.778333306312561


Epoch: 10, Loss: 0.50, Accuracy: 0.87
TESTING:
Val Accuracy: 0.79
F1 score: 0.7850000262260437


Epoch: 11, Loss: 0.42, Accuracy: 0.90
T

In [16]:
print(train_acc_list) 
print(val_acc_list) 

[0.22483850802250469, 0.4323817461971244, 0.5655344863513232, 0.6563867472390081, 0.7074390498020421, 0.7534903104813503, 0.7876640966868097, 0.8239216503438216, 0.8497603667430714, 0.8722650552198374, 0.8972702646384664, 0.9083142321316942, 0.9276932694311315, 0.9360283392373411, 0.9476974369660346, 0.9543654928110022, 0.9560325067722443, 0.9681183579912481, 0.9727026463846634, 0.972285892894353, 0.9768701812877683, 0.9812460929360284, 0.9820795999166493, 0.9818712231714941, 0.9799958324650969, 0.9818712231714941, 0.983538237132736, 0.987080641800375, 0.9856220045842884, 0.9845801208585122]
[0.3416666666666667, 0.485, 0.5633333333333334, 0.6683333333333333, 0.6933333333333334, 0.7783333333333333, 0.7383333333333333, 0.6916666666666667, 0.7783333333333333, 0.785, 0.8183333333333334, 0.805, 0.755, 0.8066666666666666, 0.83, 0.83, 0.8016666666666666, 0.8133333333333334, 0.7933333333333333, 0.805, 0.8433333333333334, 0.8033333333333333, 0.8366666666666667, 0.825, 0.8366666666666667, 0.8366

In [ ]:
# [0.23395833333333332, 0.41291666666666665, 0.5372916666666666, 0.6233333333333333, 0.694375, 0.7441666666666666, 0.77625, 0.813125, 0.834375, 0.860625, 0.88125, 0.895, 0.9072916666666667, 0.9214583333333334, 0.9352083333333333, 0.9460416666666667, 0.9564583333333333, 0.9597916666666667, 0.963125, 0.9704166666666667, 0.9714583333333333, 0.9733333333333334, 0.978125, 0.9789583333333334, 0.980625, 0.9820833333333333, 0.985625, 0.9858333333333333, 0.9804166666666667, 0.9877083333333333]
# [0.3433333333333333, 0.42833333333333334, 0.5866666666666667, 0.68, 0.67, 0.74, 0.74, 0.7733333333333333, 0.8016666666666666, 0.7733333333333333, 0.7966666666666666, 0.7866666666666666, 0.83, 0.81, 0.8466666666666667, 0.8183333333333334, 0.8266666666666667, 0.8183333333333334, 0.8483333333333334, 0.87, 0.835, 0.8116666666666666, 0.8566666666666667, 0.8583333333333333, 0.8316666666666667, 0.8483333333333334, 0.8366666666666667, 0.8116666666666666, 0.8433333333333334, 0.8216666666666667]

# Results:

---

Sample Length (s) : 60

lr= 10**(-5)

Model: Resnet18

Normalization: Yes

Number of classes: 12

Mini-batch size = 1

Number of Samples in training: 2000*(0.8)

`fs = [50, 8000]`

Best Val Accurary = 0.69 epoch 14/30 $|$ 0.81 epoch 17

Input shape: `(768, 6025)`


Trainable params: 11,181,000


---

Sample Length (s) : 60

lr= 10**(-5)

Model: Resnet 50

Normalization: Yes

Number of classes: 12

Mini-batch size = 1

Number of Samples in training: 2000*(0.8)

`fs = [50, 8000]`

Best Val Accurary = 0.51 epoch 11/15

Input shape: `(768, 6025)`


Trainable params: 23,559,180


# Testing

how to create confusion matrix: https://christianbernecker.medium.com/how-to-create-a-confusion-matrix-in-pytorch-38d06a7f04b7

In [14]:
model = ResNet(3, ResBlock, [2, 2, 2, 2], useBottleneck=False, outputs=12) # resnet18
device = torch.device("cuda:0" if torch.cuda.is_available() else  "cpu") #
model.load_state_dict(torch.load("/content/gdrive/MyDrive/BestResnet18.pt"))
model = model.to(device)

In [15]:
inference(model, test_dl)

Val Accuracy: 0.81
F1 score: 0.809999942779541


(tensor(0.8100), 0.81)